<a href="https://colab.research.google.com/github/SumiranRai/MDSC-Lab/blob/main/MDSC-205-Software-Lab-In-Data-Engineering/SQL_in_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sumiran Rai

Regd. No. - 24040208007

Software Lab In Data Engineering

# SQL with PySpark

Installing PySpark

In [3]:
!pip install pyspark

Creating a Spark Session and Loading the Datasets into Spark Dataframes

In [7]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("EmployeeData").getOrCreate()

# Read CSV file into a DataFrame with header and inferred schema
employees= spark.read.csv("/content/employees.csv", header=True, inferSchema=True)

# Read CSV file into a DataFrame with header and inferred schema
departments = spark.read.csv("/content/departments.csv", header=True, inferSchema=True)


Display the first few rows of the DataFrame for verification

In [8]:
employees.show(5)

+---+------+---+----------+------+----------+
| id|  name|age|department|salary|experience|
+---+------+---+----------+------+----------+
|  1| Jack1| 48|        IT| 54831|        39|
|  2|  Ivy2| 26|        HR| 99355|        24|
|  3|David3| 22|        IT| 85694|        12|
|  4|  Eva4| 59|   Support|113439|        24|
|  5|Frank5| 31|        HR| 65103|        22|
+---+------+---+----------+------+----------+
only showing top 5 rows



Display the first few rows of the DataFrame for verification

In [9]:
departments.show(5)

+----------+-------------+
|department|     location|
+----------+-------------+
|        HR|     New York|
|        IT|San Francisco|
|   Finance|       London|
| Marketing|        Paris|
|     Sales|       Berlin|
+----------+-------------+
only showing top 5 rows



Print the schema to verify column data types

In [10]:
employees.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- experience: integer (nullable = true)



Create or replace temporary views for the 'employees' and 'departments' DataFrames.

These views allow SQL queries to be executed on the DataFrames within the current Spark session.

In [11]:
employees.createOrReplaceTempView("employees")
departments.createOrReplaceTempView("departments")

Returns the rows where Salary > 60000 and Name contain 'D'

In [12]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("SparkSQLDemo").getOrCreate()

# Load CSV into DataFrame
df = spark.read.csv("employees.csv", header=True, inferSchema=True)

# Register DataFrame as SQL Table
df.createOrReplaceTempView("employees")

# Run SQL Query
result = spark.sql("SELECT * FROM employees WHERE salary > 60000 and name like '%D%'")
result.show()


+---+--------+---+----------+------+----------+
| id|    name|age|department|salary|experience|
+---+--------+---+----------+------+----------+
|  3|  David3| 22|        IT| 85694|        12|
| 22| David22| 23| Marketing| 60725|        37|
| 27| David27| 59|   Finance|100104|        26|
| 28| David28| 37|     Sales| 77333|        31|
| 50| David50| 37| Marketing| 85070|        35|
| 58| David58| 31|        HR| 71062|        12|
| 69| David69| 57|     Sales| 94519|        14|
| 74| David74| 56|        HR| 65828|         4|
| 80| David80| 28|        IT| 98780|        23|
| 91| David91| 44|        IT| 65134|        40|
|123|David123| 27| Marketing| 61655|         1|
|190|David190| 49| Marketing| 86555|        21|
+---+--------+---+----------+------+----------+



### Performing an Inner Join

Definition - Returns only matching rows from both tables.
If there is no match, the row is excluded.  

Explanation:

Retrieves employees who have a matching department in the departments table.
If an employee's department is missing from departments, they are excluded.

In [13]:
# Load departments dataset
dept_df = spark.read.csv("departments.csv", header=True, inferSchema=True)

# Register DataFrame as SQL Table
dept_df.createOrReplaceTempView("departments")

# Perform an INNER JOIN
join_result = spark.sql("""
    SELECT e.name, e.department, d.location
    FROM employees e
    INNER JOIN departments d
    ON e.department = d.department
""")

join_result.show()

+---------+-----------+-------------+
|     name| department|     location|
+---------+-----------+-------------+
|    Jack1|         IT|San Francisco|
|     Ivy2|         HR|     New York|
|   David3|         IT|San Francisco|
|     Eva4|    Support|      Chicago|
|   Frank5|         HR|     New York|
|   Alice6|  Marketing|        Paris|
|   Alice7|  Marketing|        Paris|
|   Alice8|    Support|      Chicago|
|   Frank9|Engineering|      Seattle|
|Charlie10|    Finance|       London|
|Charlie11|         HR|     New York|
|  Grace12|    Finance|       London|
|    Ivy13|    Finance|       London|
|  Grace14|         HR|     New York|
|    Ivy15|         IT|San Francisco|
|  Alice16|      Sales|       Berlin|
|Charlie17|    Support|      Chicago|
|  David18|      Sales|       Berlin|
|    Eva19|         HR|     New York|
|    Eva20|Engineering|      Seattle|
+---------+-----------+-------------+
only showing top 20 rows



### Performing a Left Join (Left Outer Join)

Definition - Returns all rows from the left table (employees).
If there is no match in the right table (departments), it returns NULL in those columns.

In [14]:
# Perform a LEFT JOIN using PySpark
join_result = spark.sql("""
    SELECT e.name, e.department, d.location
    FROM employees e
    LEFT JOIN departments d
    ON e.department = d.department
""")

# Show results
join_result.show()

+---------+-----------+-------------+
|     name| department|     location|
+---------+-----------+-------------+
|    Jack1|         IT|San Francisco|
|     Ivy2|         HR|     New York|
|   David3|         IT|San Francisco|
|     Eva4|    Support|      Chicago|
|   Frank5|         HR|     New York|
|   Alice6|  Marketing|        Paris|
|   Alice7|  Marketing|        Paris|
|   Alice8|    Support|      Chicago|
|   Frank9|Engineering|      Seattle|
|Charlie10|    Finance|       London|
|Charlie11|         HR|     New York|
|  Grace12|    Finance|       London|
|    Ivy13|    Finance|       London|
|  Grace14|         HR|     New York|
|    Ivy15|         IT|San Francisco|
|  Alice16|      Sales|       Berlin|
|Charlie17|    Support|      Chicago|
|  David18|      Sales|       Berlin|
|    Eva19|         HR|     New York|
|    Eva20|Engineering|      Seattle|
+---------+-----------+-------------+
only showing top 20 rows



### Performing a Right Join (Right Outer Join)

Definition - Returns all rows from the right table (departments).
If there is no match in the left table (employees), it returns NULL in those columns.

In [15]:
# Perform a RIGHT JOIN using PySpark
join_result = spark.sql("""
    SELECT e.name, e.department, d.location
    FROM employees e
    RIGHT JOIN departments d
    ON e.department = d.department
""")

# Show results
join_result.show()

+----------+----------+--------+
|      name|department|location|
+----------+----------+--------+
|   Jack199|        HR|New York|
|Charlie194|        HR|New York|
|   Jack192|        HR|New York|
|    Bob167|        HR|New York|
|  Alice155|        HR|New York|
|    Ivy153|        HR|New York|
| Hannah145|        HR|New York|
|Charlie143|        HR|New York|
|  Frank135|        HR|New York|
|   Jack132|        HR|New York|
|    Bob126|        HR|New York|
|  David109|        HR|New York|
| Hannah105|        HR|New York|
|   Jack103|        HR|New York|
| Hannah101|        HR|New York|
|   Grace96|        HR|New York|
|     Bob92|        HR|New York|
|     Bob89|        HR|New York|
|     Eva82|        HR|New York|
|   David74|        HR|New York|
+----------+----------+--------+
only showing top 20 rows



### Performing a query on a Temporary View using Spark SQL

In [16]:
# Import necessary PySpark modules
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Initialize Spark Session
spark = SparkSession.builder.appName("TempViewDemo").getOrCreate()

# Sample Data
data = [
    Row(id=1, name="Alice", age=25),
    Row(id=2, name="Bob", age=30),
    Row(id=3, name="Charlie", age=28)
]

# Create DataFrame
df = spark.createDataFrame(data)

# Create a TEMPORARY VIEW
df.createOrReplaceTempView("people_view")

# Query the temporary view using Spark SQL
result = spark.sql("SELECT * FROM people_view WHERE age > 26")

# Show Results
result.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  2|    Bob| 30|
|  3|Charlie| 28|
+---+-------+---+



### Global Temporary View in PySpark

A Global Temporary View in PySpark is similar to a Temporary View, but it persists across multiple Spark sessions within the same application.

	Queried using global_temp.<view_name>
  
Exists as long as the application is running

In [17]:
# Import necessary PySpark modules
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Initialize Spark Session
spark = SparkSession.builder.appName("GlobalTempViewDemo").getOrCreate()

# Sample Data
data = [
    Row(id=1, name="Alice", age=25),
    Row(id=2, name="Bob", age=30),
    Row(id=3, name="Charlie", age=28)
]

# Create DataFrame
df = spark.createDataFrame(data)

# Create a GLOBAL TEMPORARY VIEW
df.createOrReplaceGlobalTempView("global_people_view")

# Query the global temporary view
global_result = spark.sql("SELECT * FROM global_temp.global_people_view WHERE age < 30")

# Show Results
global_result.show()


+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|  Alice| 25|
|  3|Charlie| 28|
+---+-------+---+



Quering the global temporary view with a new Spark Session

Does not disappear when the session ends

In [18]:
# Start a new Spark session
spark2 = SparkSession.builder.appName("NewSession").getOrCreate()

# Query the global temporary view
new_result = spark2.sql("SELECT * FROM global_temp.global_people_view")

# Show Results
new_result.show()


+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|  Alice| 25|
|  2|    Bob| 30|
|  3|Charlie| 28|
+---+-------+---+



### Use createOrReplaceTempView for quick in-session analysis.

In [21]:
# Import necessary PySpark modules
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Start Spark session
spark = SparkSession.builder.appName("SessionTempViewDemo").getOrCreate()

# Sample Data
data = [Row(id=1, name="Alice", age=25), Row(id=2, name="Bob", age=31)]

# Create DataFrame
df = spark.createDataFrame(data)

# Create a SESSION TEMPORARY VIEW
df.createOrReplaceTempView("people_view")

# Query the session temporary view
session_result = spark.sql("SELECT * FROM people_view WHERE age > 26")

session_result.show()


+---+----+---+
| id|name|age|
+---+----+---+
|  2| Bob| 31|
+---+----+---+



### Use createOrReplaceGlobalTempView when you need to share the table across multiple Spark sessions.

In [22]:
# Import necessary PySpark modules
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Stop the existing Spark session
spark.stop()

# Get an existing SparkSession or, if there is no existing one, create a new one
new_spark = SparkSession.builder.appName("NewSession").getOrCreate()

# Recreate the global temporary view in the new session
# Sample Data
data = [Row(id=1, name="Alice", age=25), Row(id=2, name="Bob", age=31)]

# Create DataFrame
df = new_spark.createDataFrame(data)  # Use new_spark to create the DataFrame

# Create a SESSION TEMPORARY VIEW
df.createOrReplaceGlobalTempView("people_view") # Recreate the view

# Query the session temporary view
session_result = new_spark.sql("SELECT * FROM global_temp.people_view WHERE age > 26")

session_result.show()

+---+----+---+
| id|name|age|
+---+----+---+
|  2| Bob| 31|
+---+----+---+



### Global Temporary View

In [23]:
# Import necessary PySpark modules
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Start a completely new session
spark = SparkSession.builder.appName("NewSession").getOrCreate()

# Sample Data
data = [Row(id=1, name="Alice", age=25), Row(id=2, name="Bob", age=31)]

# Create DataFrame
df = spark.createDataFrame(data)  # Use new_spark to create the DataFrame

# Create a GLOBAL TEMPORARY VIEW
df.createOrReplaceGlobalTempView("global_people_view")

# Query the global temporary view
# Use spark to query the view since it was created in the new session
global_result = spark.sql("SELECT * FROM global_temp.global_people_view WHERE age < 30")

global_result.show()

+---+-----+---+
| id| name|age|
+---+-----+---+
|  1|Alice| 25|
+---+-----+---+



In [24]:
# Start a new Spark session
# Create a Spark session for handling large-scale data processing
spark_new = SparkSession.builder.appName("AfterRestart").getOrCreate()

# query the spark session
global_result = spark_new.sql("SELECT * FROM global_temp.global_people_view")
# Display the first few rows of the DataFrame for verification
global_result.show()


+---+-----+---+
| id| name|age|
+---+-----+---+
|  1|Alice| 25|
|  2|  Bob| 31|
+---+-----+---+



### Save DataFrame as a persistent table

Use saveAsTable if you want to store data permanently for future use.

In [25]:
# Save DataFrame as a persistent table
df.write.mode("overwrite").saveAsTable("permanent_people_tables")

# Query the saved table
permanent_result = spark.sql("SELECT * FROM permanent_people_tables WHERE age < 30")

permanent_result.show()


+---+-----+---+
| id| name|age|
+---+-----+---+
|  1|Alice| 25|
+---+-----+---+



In [26]:
# Start a new Spark session
spark_new = SparkSession.builder.appName("AfterRestart").getOrCreate()

#  This will work because the table is persisted
permanent_result = spark_new.sql("SELECT * FROM permanent_people_tables")
# Display the first few rows of the DataFrame for verification
permanent_result.show()


+---+-----+---+
| id| name|age|
+---+-----+---+
|  1|Alice| 25|
|  2|  Bob| 31|
+---+-----+---+



### Create a session temporay view, global temporary view and a persistent table

In [27]:
# Import necessary PySpark modules
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Create Spark session
spark = SparkSession.builder.appName("TestViews").getOrCreate()

# Sample data
data = [
    Row(id=1, name="Alice", age=25),
    Row(id=2, name="Bob", age=30),
    Row(id=3, name="Charlie", age=28)
]

# Create DataFrame
df = spark.createDataFrame(data)

# Create a session temporary view
df.createOrReplaceTempView("session_people_view")

# Create a global temporary view
df.createOrReplaceGlobalTempView("global_people_view")

# Create a persistent table
df.write.mode("overwrite").saveAsTable("persistent_people_tables")


#### Stop the session

In [28]:
spark.stop()

#### Start a new session

In [29]:
# Start a new session
new_spark = SparkSession.builder.appName("NewSession").getOrCreate()

# Try accessing the session temporary view (Should FAIL)
try:
    new_spark.sql("SELECT * FROM session_people_view").show()
except Exception as e:
    print("Session Temporary View: FAILED as expected!", e)

# Try accessing the global temporary view (Should WORK)
try:
    new_spark.sql("SELECT * FROM global_temp.global_people_view").show()
except Exception as e:
    print("Global Temporary View: FAILED! This should have worked.", e)

# Try accessing the persistent table (Should WORK)
try:
    new_spark.sql("SELECT * FROM persistent_people_tables").show()
except Exception as e:
    print("Persistent Table: FAILED! This should have worked.", e)


Session Temporary View: FAILED as expected! [TABLE_OR_VIEW_NOT_FOUND] The table or view `session_people_view` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [session_people_view], [], false

Global Temporary View: FAILED! This should have worked. [TABLE_OR_VIEW_NOT_FOUND] The table or view `global_temp`.`global_people_view` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [global_temp, globa

In [30]:
# Import necessary PySpark modules
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder.appName("EmployeeData").getOrCreate()

In [31]:
# Import necessary PySpark modules
from pyspark.sql.functions import expr

data = [
    ("Alice", ["Python", "SQL"]),
    ("Bob", ["Java", "Scala"]),
    ("Charlie", ["C++", "R"])
]
columns = ["name", "skills"]

# Create DataFrame
skills_df = spark.createDataFrame(data, columns)


In [32]:
skills_df.show()

+-------+-------------+
|   name|       skills|
+-------+-------------+
|  Alice|[Python, SQL]|
|    Bob|[Java, Scala]|
|Charlie|     [C++, R]|
+-------+-------------+



In [33]:
skills_df = skills_df.withColumn(
    "uppercase_skills",
    expr("transform(skills, x -> upper(x))")
)

skills_df.show(truncate=False)


+-------+-------------+----------------+
|name   |skills       |uppercase_skills|
+-------+-------------+----------------+
|Alice  |[Python, SQL]|[PYTHON, SQL]   |
|Bob    |[Java, Scala]|[JAVA, SCALA]   |
|Charlie|[C++, R]     |[C++, R]        |
+-------+-------------+----------------+



In [34]:
# Import necessary PySpark modules
from pyspark.sql.functions import expr
data = [
    ("Alice", ["Python", "SQL"]),
    ("Bob", ["Java", "Scala"]),
    ("Charlie", ["C++", "R"])
]
columns = ["name", "skills"]

# Create DataFrame
skills_df = spark.createDataFrame(data, columns)

In [35]:
skills_df = skills_df.withColumn(
    "prefixed_skills",
    expr("transform(skills, x -> concat('Skill: ', x))")
)

skills_df.show(truncate=False)

+-------+-------------+---------------------------+
|name   |skills       |prefixed_skills            |
+-------+-------------+---------------------------+
|Alice  |[Python, SQL]|[Skill: Python, Skill: SQL]|
|Bob    |[Java, Scala]|[Skill: Java, Skill: Scala]|
|Charlie|[C++, R]     |[Skill: C++, Skill: R]     |
+-------+-------------+---------------------------+



In [36]:
skills_df = skills_df.withColumn(
    "skill_lengths",
    expr("transform(skills, x -> length(x))")
)

# Display the first few rows of the DataFrame for verification
skills_df.show(truncate=False)


+-------+-------------+---------------------------+-------------+
|name   |skills       |prefixed_skills            |skill_lengths|
+-------+-------------+---------------------------+-------------+
|Alice  |[Python, SQL]|[Skill: Python, Skill: SQL]|[6, 3]       |
|Bob    |[Java, Scala]|[Skill: Java, Skill: Scala]|[4, 5]       |
|Charlie|[C++, R]     |[Skill: C++, Skill: R]     |[3, 1]       |
+-------+-------------+---------------------------+-------------+



In [37]:
skills_df = skills_df.withColumn(
    "skill_lengths",
    expr("transform(skills, x -> CASE WHEN x = 'Python' THEN upper(x) ELSE x END)")
)

skills_df.show(truncate=False)

+-------+-------------+---------------------------+-------------+
|name   |skills       |prefixed_skills            |skill_lengths|
+-------+-------------+---------------------------+-------------+
|Alice  |[Python, SQL]|[Skill: Python, Skill: SQL]|[PYTHON, SQL]|
|Bob    |[Java, Scala]|[Skill: Java, Skill: Scala]|[Java, Scala]|
|Charlie|[C++, R]     |[Skill: C++, Skill: R]     |[C++, R]     |
+-------+-------------+---------------------------+-------------+



In [38]:
data = [
    ("Alice", ["Python", "SQL"]),
    ("Bob", ["Java", "Scala"]),
    ("Charlie", ["C++", "R"])
]
columns = ["name", "skills"]

In [39]:
skills_df = skills_df.withColumn(
    "skill_lengths",
    expr("transform(skills, x -> CASE WHEN x = 'Python' THEN upper(x) ELSE x END)")
)

skills_df.show(truncate=False)

+-------+-------------+---------------------------+-------------+
|name   |skills       |prefixed_skills            |skill_lengths|
+-------+-------------+---------------------------+-------------+
|Alice  |[Python, SQL]|[Skill: Python, Skill: SQL]|[PYTHON, SQL]|
|Bob    |[Java, Scala]|[Skill: Java, Skill: Scala]|[Java, Scala]|
|Charlie|[C++, R]     |[Skill: C++, Skill: R]     |[C++, R]     |
+-------+-------------+---------------------------+-------------+

